In [7]:
from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key

In [5]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)
# name = llm.predict("Generate some indian food items with cucomber")
# print(name)

## Prompt Templates

In [7]:
from langchain.prompts import PromptTemplate
# prompt_template_name = PromptTemplate(
#     input_variables=['vegetable'],
#     template = "Generate food items with {vegetable}. Suggest indian food items"
# )
# promptt = prompt_template_name.format(vegetable="apple")
# items = llm.predict(promptt)
# print(items)

## Chains with llm

In [9]:
from langchain.chains import LLMChain

# chain = LLMChain(llm=llm, prompt =prompt_template_name)

In [12]:
print(chain.run("banana"))



1. Banana Chutney 
2. Banana Halwa 
3. Banana Lassi 
4. Banana Kheer 
5. Banana Fry 
6. Banana Poori 
7. Banana Appam 
8. Banana Roti 
9. Banana Pakora 
10. Banana Kofta Curry


#### Simple Sequential Chain

Single Input and One output

In [14]:
recepi_prompt = PromptTemplate(
    input_variables=['foodItem'],
    template=f"""
Your task is generate food recipe step by 
step below format with give food=`{{foodItem}}`
"""
)
p = recepi_prompt.format(foodItem="Italian")
print(p)


Your task is generate food recipe step by 
step below format with give food=Italian



In [27]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

foodItems_prompt = PromptTemplate(
    input_variables=['vegetable'],
    template='Generate only one indian food item with {vegetable}'
)
nameChain = LLMChain(llm=llm, prompt=foodItems_prompt)

recepi_prompt = PromptTemplate(
    input_variables=['foodItem'],
    template=f"""
Your task is generate food recipe step by 
step below format with give food=`{{foodItem}}`
"""
)
recepiPrompt = LLMChain(llm=llm, prompt=recepi_prompt)


In [28]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [nameChain, recepiPrompt])

content = chain.run("Banana")
print(content)


1. Peel and chop 2 ripe bananas into small pieces.

2. Place chopped banana in a bowl and add 2 cups of plain yogurt.

3. Add 1/2 teaspoon of ground cardamom and 1/4 teaspoon of ground cinnamon.

4. Stir until all the ingredients are combined.

5. Add 1 tablespoon of honey or sugar (optional).

6. Refrigerate until ready to serve.

7. Serve chilled with your favorite meal. Enjoy!


#### Sequential Chain

Muktiple Input and moultiple Output

In [29]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

foodItems_prompt = PromptTemplate(
    input_variables=['vegetable'],
    template='Generate only one indian food item with {vegetable}'
)
nameChain = LLMChain(llm=llm, prompt=foodItems_prompt, output_key="foodItem")

recepi_prompt = PromptTemplate(
    input_variables=['foodItem'],
    template=f"""
Your task is generate food recipe step by 
step below format with give food=`{{foodItem}}`
"""
)
recepiPrompt = LLMChain(llm=llm, prompt=recepi_prompt, output_key="recipe")


In [33]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [nameChain, recepiPrompt],
    input_variables = ['vegetable'],
    output_variables = ['foodItem', "recipe"]
)
content = chain("Banana")
print(content)

{'vegetable': 'Banana', 'foodItem': '\n\nBanana Dosa', 'recipe': '\n1. Peel and mash 3-4 ripe bananas.\n2. In a bowl mix the mashed banana with a pinch of salt, ½ teaspoon of cumin powder, ½ teaspoon of chili powder and 1 teaspoon of oil.\n3. Add ¾ to 1 cup of rice flour.\n4. Gradually add water and mix everything into a batter of medium thick consistency.\n5. Heat a flat non-stick pan on medium heat and pour a ladle full of batter.\n6. Spread the batter evenly to form a thin dosa.\n7. Sprinkle a few drops of oil and cook the dosa on medium-low heat till it turns golden brown.\n8. Flip the dosa and cook the other side for a few seconds.\n9. Serve the banana dosa hot with chutney or sambar.'}


In [36]:
print(content)

{'vegetable': 'Banana', 'foodItem': '\n\nBanana Dosa', 'recipe': '\n1. Peel and mash 3-4 ripe bananas.\n2. In a bowl mix the mashed banana with a pinch of salt, ½ teaspoon of cumin powder, ½ teaspoon of chili powder and 1 teaspoon of oil.\n3. Add ¾ to 1 cup of rice flour.\n4. Gradually add water and mix everything into a batter of medium thick consistency.\n5. Heat a flat non-stick pan on medium heat and pour a ladle full of batter.\n6. Spread the batter evenly to form a thin dosa.\n7. Sprinkle a few drops of oil and cook the dosa on medium-low heat till it turns golden brown.\n8. Flip the dosa and cook the other side for a few seconds.\n9. Serve the banana dosa hot with chutney or sambar.'}


In [44]:
step =content['recipe'].strip().split("\n")

In [47]:
for item in step:
    print("--",item)

-- 1. Peel and mash 3-4 ripe bananas.
-- 2. In a bowl mix the mashed banana with a pinch of salt, ½ teaspoon of cumin powder, ½ teaspoon of chili powder and 1 teaspoon of oil.
-- 3. Add ¾ to 1 cup of rice flour.
-- 4. Gradually add water and mix everything into a batter of medium thick consistency.
-- 5. Heat a flat non-stick pan on medium heat and pour a ladle full of batter.
-- 6. Spread the batter evenly to form a thin dosa.
-- 7. Sprinkle a few drops of oil and cook the dosa on medium-low heat till it turns golden brown.
-- 8. Flip the dosa and cook the other side for a few seconds.
-- 9. Serve the banana dosa hot with chutney or sambar.


### Agents

In [13]:
from secret_key import serpapi_key
import os
os.environ['SERPER_API_KEY'] = serpapi_key
# os.environ['OPENAI_API_KEY'] = openapi_key

serpapi and llm-math tool

In [11]:
print(serpapi_key)

5aad7363c865e81b95ef714b5b83509261326dcfb7959f69128988af1668ac24


In [17]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run("Where is the vanue of world cup cricket final match in 2023?")



> Entering new AgentExecutor chain...
 I need to find out the location of the venue
Action: Search
Action Input: "World Cup Cricket 2023 Venue"
Observation: India
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-4h1riewe2DAzat5ygUYh2tZV on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


 I now know the final answer
Final Answer: The venue of the World Cup Cricket final match in 2023 is India.

> Finished chain.


'The venue of the World Cup Cricket final match in 2023 is India.'

In [ ]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")

## Memory

In [19]:
from langchain.prompts import PromptTemplate
prompt_template_name = PromptTemplate(
    input_variables=['vegetable'],
    template = "Generate food items with {vegetable}. Suggest indian food items"
)
promptt = prompt_template_name.format(vegetable="apple")
items = llm.predict(promptt)
print(items)



1. Apple Halwa
2. Apple Kheer
3. Apple Pachadi
4. Apple Payasam
5. Apple Chutney
6. Apple Curry
7. Apple Raita
8. Apple Kebab
9. Apple Kofta
10. Apple Pulao


In [20]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt =prompt_template_name)

In [21]:
print(chain.run("banana"))



1. Banana Halwa
2. Banana Sheera
3. Banana Appam
4. Banana Payasam
5. Banana Raita
6. Banana Kofta Curry
7. Banana Chips
8. Banana Bread
9. Banana Smoothie
10. Banana Lassi


In [22]:
print(chain.run("jackfood"))



1. Jackfruit Biryani
2. Jackfruit Curry
3. Jackfruit Kebab
4. Jackfruit Tikka Masala
5. Jackfruit Pulao
6. Jackfruit Pakora
7. Jackfruit Samosa
8. Jackfruit Dal
9. Jackfruit Raita
10. Jackfruit Halwa


In [23]:
chain.memory

In [24]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [25]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Beans")
print(name)



1. Rajma Chawal
2. Chana Masala
3. Dal Makhani
4. Moong Dal Tadka
5. Matar Paneer
6. Aloo Chole
7. Chole Bhature
8. Masala Vada
9. Dal Fry
10. Vegetable Biryani


In [26]:
name = chain.run("Cucomber")
print(name)



1. Cucumber Raita
2. Cucumber Pachadi
3. Cucumber Kebab
4. Cucumber Salad
5. Cucumber Samosa
6. Cucumber Pakora
7. Cucumber Kofta Curry
8. Cucumber and Coconut Chutney
9. Cucumber and Mint Raita
10. Cucumber and Onion Bhaji


In [27]:
print(chain.memory.buffer)

Human: Beans
AI: 

1. Rajma Chawal
2. Chana Masala
3. Dal Makhani
4. Moong Dal Tadka
5. Matar Paneer
6. Aloo Chole
7. Chole Bhature
8. Masala Vada
9. Dal Fry
10. Vegetable Biryani
Human: Cucomber
AI: 

1. Cucumber Raita
2. Cucumber Pachadi
3. Cucumber Kebab
4. Cucumber Salad
5. Cucumber Samosa
6. Cucumber Pakora
7. Cucumber Kofta Curry
8. Cucumber and Coconut Chutney
9. Cucumber and Mint Raita
10. Cucumber and Onion Bhaji
